## Feedback Generation

### Generate Feedback Suggestions
Upload the json files from `data/2_submissions/json_files` to the playground. In evaluation mode, generate feedback for each exercise and export the results. Make sure that the configuration names for feedback generation do not contain underscores '_'. Also make sure that the configuration names are unique for different feedback types but the same across different exercises.

The downloaded json files should have the following naming scheme:
`text_results_<Configuration name (e.g.: LLM)>_<...>`

**Do not change the names of the downloaded files!**

Save these files in the `data/3_feedback_suggestions/feedback_suggestions` directory.

### Read Feedback Suggestions
Once you have saved the feedback suggestion files in the `data/3_feedback_suggestions/feedback_suggestions` directory, run the following cell to read the feedback suggestions from the files and to add them to the sampled submissions.

In [ ]:
from service.json_service import (
    read_result_files_to_dataframe,
    add_feedback_suggestions_to_data,
    fill_missing_feedback_with_tutor_feedback,
)
import pandas as pd

sampled_submissions = pd.read_csv("data/2_submissions/submissions.csv")
feedback_suggestions = read_result_files_to_dataframe(
    "data/3_feedback_suggestions/feedback_suggestions"
)

sampled_submissions_with_feedback = add_feedback_suggestions_to_data(
    sampled_submissions, feedback_suggestions
)
sampled_submissions_with_feedback = fill_missing_feedback_with_tutor_feedback(
    sampled_submissions_with_feedback
)

sampled_submissions_with_feedback = sampled_submissions_with_feedback.assign(
    feedback_text=None
)

### Save the Feedback Suggestions
Save the feedback suggestions to a CSV and JSON file for the next steps in the evaluation process.
You can also retrieve the feedback suggestions from an existing CSV file.

In [ ]:
from service.json_service import (
    group_exercise_data,
    exercises_to_json,
)

sampled_submissions_with_feedback.to_csv(
    "data/3_feedback_suggestions/feedback_suggestions.csv", index=False
)
# sampled_submissions_with_feedback = pd.read_csv("data/3_feedback_suggestions/feedback_suggestions.csv")

exercises = group_exercise_data(sampled_submissions_with_feedback)
exercises_to_json(exercises, "data/3_feedback_suggestions/json_files")